In [13]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold

In [2]:
import math

def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

In [3]:
#evaluation method - RMSLE
def RMSLE(model_y, actual_y) : 
    return np.sqrt(np.sum((np.log(model_y + 1) - np.log(actual_y + 1)) ** 2) / len(model_y))

In [6]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [7]:
X_train = train.iloc[ : , :-1]
y_train = train.trip_duration
X_test = test

In [8]:
#drop id
X_train = X_train.drop(labels = ["id"], axis = 1)
X_test = X_test.drop(labels = ["id"], axis = 1)

In [9]:
ls = []
for a,b,c,d in zip(X_train["pickup_latitude"], X_train["pickup_longitude"], X_train["dropoff_latitude"], X_train["dropoff_longitude"]) :
    dist = distance([a, b], [c, d])
    ls.append(dist)

    
ls2 = []
for a,b,c,d in zip(X_test["pickup_latitude"], X_test["pickup_longitude"], X_test["dropoff_latitude"], X_test["dropoff_longitude"]) :
    test_dist = distance([a, b], [c, d])
    ls2.append(test_dist)
    
X_train["dist"] = ls    
X_test["dist"] = ls2

In [10]:
X_train["hour"] = pd.to_datetime(X_train["pickup_datetime"]).dt.hour
X_test["hour"] = pd.to_datetime(X_test["pickup_datetime"]).dt.hour

In [11]:
X_train = pd.get_dummies(X_train, prefix = "vendor_id", columns = ["vendor_id", "store_and_fwd_flag"])
X_test = pd.get_dummies(X_test, prefix = "vendor_id", columns = ["vendor_id", "store_and_fwd_flag"])

In [12]:
#delete pickup_datetime, dropoff_datetime
X_train.drop(labels = ["pickup_datetime", "dropoff_datetime"], axis = 1, inplace = True)
X_test.drop(labels = ["pickup_datetime"], axis = 1, inplace = True)

In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [17]:
rf = RandomForestClassifier(n_estimators = 100, oob_score=True, random_state=2)

In [18]:
model = rf
cv = KFold(10)

In [19]:
scores = np.zeros(10)
for i, (train_index, test_index) in enumerate(cv.split(X_train)):
    X_train = X_train.iloc[train_index]
    y_train = y_train.iloc[train_index]
    X_test = X_test.iloc[test_index]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    scores[i] = RMSLE(y_pred, y_train)

MemoryError: could not allocate 13602127872 bytes